# Momentum Strategy - Weekly Rebalance


1. Rank Stocks in SP500 based on momentum. Momentum: calculated by multiplying the annualized exponential regression slope of past 90 days by R^2 coeeffecient of regression calc
2. Position Size: Calc using 20 day ATR of each stock, multiplied by 10 basis points of portfolio value
3. Open new positions only if SP 500 is above 200 day moving avg
4. Every week, sell stocks that are not in top 20% momentum ranking, or have fallen below their 100 day moving avg. Buy stocks in top 20% momentum rankings
5. Every other week, rebalance exisitng positions with updated ATR values
https://teddykoker.com/2019/05/momentum-strategy-from-stocks-on-the-move-in-python/

DATA: 
* AlphaVantage: https://www.alphavantage.co/documentation/6a772cd25
* QUANDL
* Or just learn Quant Connect
* Look into Kelly Edge

In [18]:
import requests
import pandas as pd
import os
import bs4 as bs
import numpy as np
from scipy.stats import linregress

In [8]:
def get_sp500():
    url = "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    resp = requests.get(url)
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text.strip()
        tickers.append(ticker)  
    #with open("sp500tickers.pickle","wb") as f:
    #    pickle.dump(tickers,f)  
    return tickers

In [9]:
def generate_massive_df(tickers):
    stocks = (
        (pd.concat(
            [pd.read_csv(f"data/{ticker}.csv", index_col='Date', parse_dates=True)[
                'Close'
            ].rename(ticker)
            for ticker in tickers if os.path.exists("data/{}.csv".format(ticker))],
            axis=1,
            sort=True)
        )
    )
    stocks = stocks.loc[:,~stocks.columns.duplicated()]
    return stocks

In [10]:
def momentum(closes):
    returns = np.log(closes)
    x = np.arange(len(returns))
    slope, _, rvalue, _, _ = linregress(x,returns)
    return ((1 + slope) ** 252 * (rvalue ** 2))

In [11]:
def apply_momentum(stocks, tickers):
    momentums = stocks.copy(deep=True)
    for ticker in tickers:
        if os.path.exists("data/{}.csv".format(ticker)):
            momentums[ticker] = stocks[ticker].rolling(90).apply(momentum, raw=False)
    return momentums

In [19]:
tickers = get_sp500()
stocks = generate_massive_df(tickers)

In [20]:
stocks

MMM        ABT       ABBV        ABMD         ACN  \
Date                                                                   
2004-01-02   83.370003  20.986116        NaN    7.000000   25.920000   
2004-01-05   84.959999  20.986116        NaN    6.910000   26.660000   
2004-01-06   84.699997  20.833490        NaN    6.990000   26.350000   
2004-01-07   83.239998  21.053452        NaN    7.160000   25.750000   
2004-01-08   82.680000  20.433968        NaN    7.290000   24.889999   
2004-01-09   82.400002  20.169117        NaN    7.300000   25.219999   
2004-01-12   83.169998  19.949156        NaN    7.220000   25.910000   
2004-01-13   82.879997  19.944666        NaN    7.120000   22.660000   
2004-01-14   83.699997  19.976089        NaN    7.290000   23.690001   
2004-01-15   84.300003  20.178095        NaN    7.220000   23.600000   
2004-01-16   85.480003  19.414963        NaN    7.470000   23.459999   
2004-01-20   80.410004  19.428431        NaN    7.910000   23.780001   
2004-01-21   81.750000  19.616968        NaN    7.920000   24.150000   
2004-01-22   82.110001  19.338650        NaN    8.340000   24.879999   
2004-01-23   81.959999  19.289270        NaN    8.140000   24.100000   
2004-01-26   82.470001  19.697771        NaN    8.340000   24.670000   
2004-01-27   82.099998  19.477810        NaN    8.240000   24.000000   
2004-01-28   80.000000  19.661858        NaN    7.560000   23.760000   
2004-01-29   79.519997  19.639414        NaN    7.950000   23.670000   
2004-01-30   79.089996  19.338650        NaN    7.750000   23.670000   
2004-02-02   78.000000  19.518209        NaN    7.600000   23.120001   
2004-02-03   78.900002  19.863863        NaN    7.270000   23.240000   
2004-02-04   79.449997  19.868353        NaN    7.020000   22.600000   
2004-02-05   79.000000  20.007513        NaN    6.690000   22.639999   
2004-02-06   79.809998  19.917732        NaN    7.170000   22.830000   
2004-02-09   79.389999  19.913242        NaN    7.020000   22.780001   
2004-02-10   81.250000  19.796528        NaN    7.360000   22.799999   
2004-02-11   81.400002  20.020979        NaN    7.380000   22.889999   
2004-02-12   80.980003  19.648392        NaN    7.540000   23.049999   
2004-02-13   79.680000  19.890799        NaN    7.340000   23.459999   
...                ...        ...        ...         ...         ...   
2019-08-22  160.880005  84.370003  67.519997  195.440002  196.339996   
2019-08-23  155.850006  81.930000  65.970001  189.589996  192.809998   
2019-08-26  156.119995  83.120003  66.459999  190.440002  195.470001   
2019-08-27  155.750000  83.650002  65.669998  190.059998  197.160004   
2019-08-28  157.550003  84.220001  66.070000  191.199997  196.839996   
2019-08-29  160.360001  84.820000  66.220001  192.009995  198.740005   
2019-08-30  161.720001  85.320000  65.739998  193.070007  198.169998   
2019-09-03  158.639999  84.059998  65.400002  183.639999  195.750000   
2019-09-04  160.070007  84.220001  66.379997  183.380005  197.710007   
2019-09-05  164.229996  85.900002  67.029999  190.250000  199.490005   
2019-09-06  163.940002  86.040001  67.620003  189.429993  201.119995   
2019-09-09  165.940002  84.220001  66.699997  186.809998  194.830002   
2019-09-10  168.860001  83.209999  68.180000  191.820007  189.660004   
2019-09-11  168.080002  84.519997  69.489998  191.070007  191.369995   
2019-09-12  170.580002  85.209999  70.830002  188.020004  195.520004   
2019-09-13  171.440002  84.269997  70.510002  186.410004  194.619995   
2019-09-16  169.669998  83.360001  70.580002  190.539993  194.160004   
2019-09-17  168.070007  83.580002  71.599998  190.539993  194.949997   
2019-09-18  167.440002  83.639999  71.269997  190.669998  194.130005   
2019-09-19  167.070007  83.230003  71.669998  192.320007  194.669998   
2019-09-20  166.759995  83.580002  72.389999  192.759995  193.089996   
2019-09-23  166.759995  83.160004  72.930000  186.009995  192.839996   
2019-09-24  164.740005  82.440002  72.949997  184.350006  1

In [37]:
tickers = get_sp500()
stocks = generate_massive_df(tickers)
stocks

MMM        ABT       ABBV        ABMD         ACN  \
Date                                                                   
2004-01-02   83.370003  20.986116        NaN    7.000000   25.920000   
2004-01-05   84.959999  20.986116        NaN    6.910000   26.660000   
2004-01-06   84.699997  20.833490        NaN    6.990000   26.350000   
2004-01-07   83.239998  21.053452        NaN    7.160000   25.750000   
2004-01-08   82.680000  20.433968        NaN    7.290000   24.889999   
2004-01-09   82.400002  20.169117        NaN    7.300000   25.219999   
2004-01-12   83.169998  19.949156        NaN    7.220000   25.910000   
2004-01-13   82.879997  19.944666        NaN    7.120000   22.660000   
2004-01-14   83.699997  19.976089        NaN    7.290000   23.690001   
2004-01-15   84.300003  20.178095        NaN    7.220000   23.600000   
2004-01-16   85.480003  19.414963        NaN    7.470000   23.459999   
2004-01-20   80.410004  19.428431        NaN    7.910000   23.780001   
2004-01-21   81.750000  19.616968        NaN    7.920000   24.150000   
2004-01-22   82.110001  19.338650        NaN    8.340000   24.879999   
2004-01-23   81.959999  19.289270        NaN    8.140000   24.100000   
2004-01-26   82.470001  19.697771        NaN    8.340000   24.670000   
2004-01-27   82.099998  19.477810        NaN    8.240000   24.000000   
2004-01-28   80.000000  19.661858        NaN    7.560000   23.760000   
2004-01-29   79.519997  19.639414        NaN    7.950000   23.670000   
2004-01-30   79.089996  19.338650        NaN    7.750000   23.670000   
2004-02-02   78.000000  19.518209        NaN    7.600000   23.120001   
2004-02-03   78.900002  19.863863        NaN    7.270000   23.240000   
2004-02-04   79.449997  19.868353        NaN    7.020000   22.600000   
2004-02-05   79.000000  20.007513        NaN    6.690000   22.639999   
2004-02-06   79.809998  19.917732        NaN    7.170000   22.830000   
2004-02-09   79.389999  19.913242        NaN    7.020000   22.780001   
2004-02-10   81.250000  19.796528        NaN    7.360000   22.799999   
2004-02-11   81.400002  20.020979        NaN    7.380000   22.889999   
2004-02-12   80.980003  19.648392        NaN    7.540000   23.049999   
2004-02-13   79.680000  19.890799        NaN    7.340000   23.459999   
...                ...        ...        ...         ...         ...   
2019-08-21  161.149994  84.900002  67.540001  195.839996  195.729996   
2019-08-22  160.880005  84.370003  67.519997  195.440002  196.339996   
2019-08-23  155.850006  81.930000  65.970001  189.589996  192.809998   
2019-08-26  156.119995  83.120003  66.459999  190.440002  195.470001   
2019-08-27  155.750000  83.650002  65.669998  190.059998  197.160004   
2019-08-28  157.550003  84.220001  66.070000  191.199997  196.839996   
2019-08-29  160.360001  84.820000  66.220001  192.009995  198.740005   
2019-08-30  161.720001  85.320000  65.739998  193.070007  198.169998   
2019-09-03  158.639999  84.059998  65.400002  183.639999  195.750000   
2019-09-04  160.070007  84.220001  66.379997  183.380005  197.710007   
2019-09-05  164.229996  85.900002  67.029999  190.250000  199.490005   
2019-09-06  163.940002  86.040001  67.620003  189.429993  201.119995   
2019-09-09  165.940002  84.220001  66.699997  186.809998  194.830002   
2019-09-10  168.860001  83.209999  68.180000  191.820007  189.660004   
2019-09-11  168.080002  84.519997  69.489998  191.070007  191.369995   
2019-09-12  170.580002  85.209999  70.830002  188.020004  195.520004   
2019-09-13  171.440002  84.269997  70.510002  186.410004  194.619995   
2019-09-16  169.669998  83.360001  70.580002  190.539993  194.160004   
2019-09-17  168.070007  83.580002  71.599998  190.539993  194.949997   
2019-09-18  167.440002  83.639999  71.269997  190.669998  194.130005   
2019-09-19  167.070007  83.230003  71.669998  192.320007  194.669998   
2019-09-20  166.759995  83.580002  72.389999  192.759995  193.089996   
2019-09-23  166.759995  83.160004  72.930000  186.009995  1

In [ ]:
momentum = apply_momentum(stocks, tickers)

In [ ]:
# Only want to calc momentum for most recent 90 days
# How tf to do dis